# tf.data API 数据流水线构建

`tf.data` 是 TensorFlow 中构建高性能数据输入流水线的核心 API。本教程系统讲解数据集创建、转换操作和性能优化策略。

## 核心知识点

1. Dataset 对象的创建方法与适用场景
2. 链式转换操作的执行语义
3. 数据打乱机制与缓冲区策略
4. 性能优化：并行化与预取

In [ ]:
import tensorflow as tf
import numpy as np

# 固定随机种子确保结果可复现
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

print(f"TensorFlow: {tf.__version__}")
print(f"Eager 执行: {tf.executing_eagerly()}")

## 1. 数据集创建

### 1.1 from_tensor_slices：内存数据

将内存中的张量沿第一维度切分，每个切片成为独立样本。

**适用场景**：
- 数据量 < 内存容量的 1/4
- 快速原型验证
- 小规模实验

**注意**：此方法会将数据复制到 TensorFlow 内存空间，大数据集应使用流式加载。

In [ ]:
# 从一维张量创建
data = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(data)

# 检查数据集规格
print(f"元素规格: {dataset.element_spec}")
print(f"数据类型: {dataset.element_spec.dtype}")
print(f"元素形状: {dataset.element_spec.shape}")
print()

# 遍历数据集
print("数据集内容:", [x.numpy() for x in dataset])

In [ ]:
# 从特征-标签对创建（监督学习场景）
features = np.random.randn(100, 5).astype(np.float32)
labels = np.random.randint(0, 3, size=(100,))

# 元组形式：(features, labels)
paired_dataset = tf.data.Dataset.from_tensor_slices((features, labels))

print(f"配对数据集规格:")
print(f"  特征: {paired_dataset.element_spec[0]}")
print(f"  标签: {paired_dataset.element_spec[1]}")
print()

# 取样验证
for feat, lbl in paired_dataset.take(2):
    print(f"特征: {feat.numpy()[:3]}..., 标签: {lbl.numpy()}")

In [ ]:
# 从字典创建（多输入模型场景）
dict_dataset = tf.data.Dataset.from_tensor_slices({
    "numeric": np.random.randn(50, 3).astype(np.float32),
    "categorical": np.random.randint(0, 10, size=(50,)),
    "label": np.random.randint(0, 2, size=(50,)),
})

print("字典数据集规格:")
for key, spec in dict_dataset.element_spec.items():
    print(f"  {key}: {spec}")

### 1.2 Dataset.range：整数序列

直接创建整数序列，无需先构建张量。常用于索引生成或简单测试。

In [ ]:
# 类似 Python range()
range_ds = tf.data.Dataset.range(5)  # 0, 1, 2, 3, 4
print("range(5):", [x.numpy() for x in range_ds])

# 指定起止和步长
range_ds2 = tf.data.Dataset.range(10, 20, 2)  # 10, 12, 14, 16, 18
print("range(10, 20, 2):", [x.numpy() for x in range_ds2])

## 2. 核心转换操作

### 2.1 map：元素级转换

`map(func)` 对每个元素应用转换函数。

**关键参数**：
- `num_parallel_calls`: 并行处理线程数，推荐 `tf.data.AUTOTUNE`
- `deterministic`: 是否保证输出顺序（默认 True）

In [ ]:
# 简单数值变换
def square_plus_one(x):
    """计算 x^2 + 1"""
    return x ** 2 + 1

data = tf.data.Dataset.range(6)
mapped = data.map(square_plus_one, num_parallel_calls=tf.data.AUTOTUNE)

print("原始:", [x.numpy() for x in data])
print("变换:", [x.numpy() for x in mapped])

In [ ]:
# 多步变换链
pipeline = (
    tf.data.Dataset.range(5)
    .map(lambda x: x * 2)                    # 乘以 2
    .map(lambda x: tf.cast(x, tf.float32))   # 转浮点
    .map(lambda x: x / 10.0)                 # 归一化
)

print("链式 map:", [f"{x.numpy():.2f}" for x in pipeline])

In [ ]:
# 处理特征-标签对
def preprocess(features, label):
    """特征标准化 + 标签转独热"""
    # 假设特征已知均值和标准差
    normalized = (features - 0.0) / 1.0
    one_hot = tf.one_hot(label, depth=3)
    return normalized, one_hot

processed = paired_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

for feat, lbl in processed.take(1):
    print(f"归一化特征: {feat.numpy()[:3]}...")
    print(f"独热标签: {lbl.numpy()}")

### 2.2 batch：批处理

`batch(batch_size)` 将连续元素组合成批次张量。

**参数**：
- `batch_size`: 批次大小
- `drop_remainder`: 是否丢弃最后不完整批次（默认 False）

In [ ]:
data = tf.data.Dataset.range(10)

# 默认保留不完整批次
batched = data.batch(3)
print("batch(3), drop_remainder=False:")
for batch in batched:
    print(f"  形状 {batch.shape}: {batch.numpy()}")

print()

# 丢弃不完整批次（训练时常用）
batched_dropped = data.batch(3, drop_remainder=True)
print("batch(3), drop_remainder=True:")
for batch in batched_dropped:
    print(f"  形状 {batch.shape}: {batch.numpy()}")

### 2.3 repeat：数据集重复

`repeat(count)` 将数据集重复指定次数。

- `count=None` 或不指定：无限重复
- `count=n`：重复 n 次

In [ ]:
base = tf.data.Dataset.range(3)

# 重复 2 次
repeated = base.repeat(2)
print("repeat(2):", [x.numpy() for x in repeated])
print(f"总元素数: {sum(1 for _ in repeated)}")

### 2.4 操作顺序的影响

`repeat` 和 `batch` 的调用顺序会产生不同结果：

- **先 repeat 后 batch**：重复的数据会跨批次边界
- **先 batch 后 repeat**：每个批次独立重复

In [ ]:
data = tf.data.Dataset.range(5)

# 先 repeat 后 batch
print("repeat(2) -> batch(3):")
result1 = data.repeat(2).batch(3)
for i, batch in enumerate(result1):
    print(f"  批次 {i}: {batch.numpy()}")

print()

# 先 batch 后 repeat
print("batch(3) -> repeat(2):")
result2 = data.batch(3).repeat(2)
for i, batch in enumerate(result2):
    print(f"  批次 {i}: {batch.numpy()}")

### 2.5 filter：条件过滤

`filter(predicate)` 保留满足条件的元素。

In [ ]:
data = tf.data.Dataset.range(20)

# 保留偶数
even = data.filter(lambda x: x % 2 == 0)
print("偶数:", [x.numpy() for x in even])

# 组合条件
combined = data.filter(lambda x: (x % 2 == 0) & (x > 10))
print("偶数且>10:", [x.numpy() for x in combined])

### 2.6 unbatch：解除批处理

`unbatch()` 将批次拆分为单个元素。

In [ ]:
batched = tf.data.Dataset.range(10).batch(4)
print("批处理后:")
for batch in batched:
    print(f"  {batch.numpy()}")

# 解除批处理
unbatched = batched.unbatch()
print("\n解除批处理:", [x.numpy() for x in unbatched])

## 3. 数据打乱机制

### 3.1 shuffle 工作原理

`shuffle(buffer_size)` 使用固定大小缓冲区进行随机打乱：

1. 从数据源填充 `buffer_size` 个元素到缓冲区
2. 从缓冲区随机选取一个元素输出
3. 从数据源取下一个元素填补空位
4. 重复直到数据源耗尽

**关键洞察**：`buffer_size` 决定打乱程度
- `buffer_size=1`：无打乱（顺序输出）
- `buffer_size=N`（N=数据集大小）：完全打乱
- `buffer_size` 介于两者之间：局部打乱

In [ ]:
data = tf.data.Dataset.range(10)
print("原始:", [x.numpy() for x in data])

# 不同 buffer_size 的效果
for buf_size in [1, 3, 5, 10]:
    shuffled = data.shuffle(buffer_size=buf_size, seed=42)
    result = [x.numpy() for x in shuffled]
    print(f"buffer_size={buf_size:2d}: {result}")

### 3.2 reshuffle_each_iteration 参数

控制每次遍历数据集时是否重新打乱：

- `True`（默认）：每个 epoch 顺序不同
- `False`：所有 epoch 顺序相同

In [ ]:
data = tf.data.Dataset.range(5)

# 默认：每次迭代重新打乱
shuffled_default = data.shuffle(5, seed=42, reshuffle_each_iteration=True).repeat(2)
result = [x.numpy() for x in shuffled_default]
print(f"reshuffle=True:")
print(f"  Epoch 1: {result[:5]}")
print(f"  Epoch 2: {result[5:]}")

# 固定顺序
shuffled_fixed = data.shuffle(5, seed=42, reshuffle_each_iteration=False).repeat(2)
result = [x.numpy() for x in shuffled_fixed]
print(f"\nreshuffle=False:")
print(f"  Epoch 1: {result[:5]}")
print(f"  Epoch 2: {result[5:]}")

### 3.3 推荐的操作顺序

**标准训练流水线顺序**：

```
shuffle -> map -> batch -> prefetch
```

**原因**：
1. `shuffle` 在 `batch` 前：确保批次内样本随机
2. `map` 在 `batch` 前：逐样本预处理（若可向量化则可后置）
3. `prefetch` 在最后：流水线优化

In [ ]:
# 标准训练流水线示例
data = tf.data.Dataset.range(20)

train_pipeline = (
    data
    .shuffle(buffer_size=20, seed=42)
    .map(lambda x: tf.cast(x, tf.float32) / 20.0, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(4, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)

print("训练流水线输出:")
for i, batch in enumerate(train_pipeline.take(3)):
    print(f"  批次 {i}: {batch.numpy()}")

## 4. 数据集切片与合并

### 4.1 take 和 skip

In [ ]:
data = tf.data.Dataset.range(20)

# take: 取前 n 个
first_five = data.take(5)
print("take(5):", [x.numpy() for x in first_five])

# skip: 跳过前 n 个
skip_five = data.skip(5)
print("skip(5):", [x.numpy() for x in skip_five])

# 组合实现切片
middle = data.skip(5).take(5)  # 相当于 [5:10]
print("skip(5).take(5):", [x.numpy() for x in middle])

### 4.2 concatenate：数据集合并

In [ ]:
ds1 = tf.data.Dataset.range(5)
ds2 = tf.data.Dataset.range(5, 10)

# 顺序合并
combined = ds1.concatenate(ds2)
print("concatenate:", [x.numpy() for x in combined])

### 4.3 zip：多数据集配对

In [ ]:
features_ds = tf.data.Dataset.from_tensor_slices(
    np.random.randn(5, 3).astype(np.float32)
)
labels_ds = tf.data.Dataset.from_tensor_slices([0, 1, 0, 1, 0])

# 配对
zipped = tf.data.Dataset.zip((features_ds, labels_ds))

print("zip 配对:")
for feat, lbl in zipped:
    print(f"  特征: {feat.numpy()[:2]}..., 标签: {lbl.numpy()}")

## 5. 性能优化

### 5.1 prefetch：预取优化

`prefetch(buffer_size)` 允许数据准备和模型计算并行执行：

- GPU 处理当前批次时，CPU 同时准备下一批次
- 有效隐藏数据加载延迟
- 推荐使用 `tf.data.AUTOTUNE` 自动调整

In [ ]:
# 完整优化流水线
def simulate_preprocess(x):
    """模拟耗时预处理"""
    return tf.cast(x, tf.float32) / 100.0

optimized = (
    tf.data.Dataset.range(100)
    .shuffle(buffer_size=100)
    .map(simulate_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
    .prefetch(tf.data.AUTOTUNE)
)

print("优化流水线:")
for batch in optimized.take(3):
    print(f"  形状: {batch.shape}, 范围: [{batch.numpy().min():.2f}, {batch.numpy().max():.2f}]")

### 5.2 interleave：并行读取多文件

从多个数据源交错读取，提高 I/O 吞吐量。

**关键参数**：
- `cycle_length`: 同时读取的数据源数量
- `num_parallel_calls`: 并行处理线程数
- `deterministic`: False 可提升性能但结果不可复现

In [ ]:
# 模拟多文件读取
def generate_file_data(file_id):
    """模拟从文件读取数据"""
    start = file_id * 100
    return tf.data.Dataset.range(start, start + 5)

# 3 个"文件"
file_ids = tf.data.Dataset.range(3)

# 交错读取
interleaved = file_ids.interleave(
    generate_file_data,
    cycle_length=3,
    num_parallel_calls=tf.data.AUTOTUNE
)

print("interleave 结果:", [x.numpy() for x in interleaved])
print("注意: 元素来自不同数据源交替出现")

## 6. 完整训练流水线示例

In [ ]:
# 模拟分类任务数据
np.random.seed(42)
NUM_SAMPLES = 1000
NUM_FEATURES = 10
NUM_CLASSES = 5

X_train = np.random.randn(NUM_SAMPLES, NUM_FEATURES).astype(np.float32)
y_train = np.random.randint(0, NUM_CLASSES, size=(NUM_SAMPLES,))

# 预处理函数
def normalize_and_onehot(features, label):
    """特征标准化 + 标签独热编码"""
    normalized = (features - tf.reduce_mean(features)) / tf.math.reduce_std(features)
    one_hot = tf.one_hot(label, depth=NUM_CLASSES)
    return normalized, one_hot

# 构建完整流水线
BATCH_SIZE = 32
BUFFER_SIZE = 1000

train_dataset = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(buffer_size=BUFFER_SIZE)
    .map(normalize_and_onehot, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

# 验证
print("训练数据集规格:")
print(f"  特征: {train_dataset.element_spec[0]}")
print(f"  标签: {train_dataset.element_spec[1]}")
print()

for features, labels in train_dataset.take(1):
    print(f"特征批次: {features.shape}")
    print(f"标签批次: {labels.shape}")
    print(f"特征统计: 均值={features.numpy().mean():.4f}, 标准差={features.numpy().std():.4f}")

## 总结

### 核心操作速查

| 操作 | 用途 | 关键参数 |
|------|------|----------|
| `from_tensor_slices()` | 内存数据创建 | data |
| `range()` | 整数序列 | start, stop, step |
| `map()` | 元素级转换 | `num_parallel_calls=AUTOTUNE` |
| `filter()` | 条件过滤 | predicate |
| `shuffle()` | 随机打乱 | `buffer_size` |
| `batch()` | 批处理 | `drop_remainder` |
| `repeat()` | 数据集重复 | count |
| `prefetch()` | 预取优化 | `AUTOTUNE` |
| `interleave()` | 并行读取 | `cycle_length` |

### 最佳实践

1. **操作顺序**: `shuffle -> map -> batch -> prefetch`
2. **并行化**: 始终使用 `num_parallel_calls=tf.data.AUTOTUNE`
3. **预取**: 流水线末尾添加 `prefetch(tf.data.AUTOTUNE)`
4. **大数据集**: 使用 `interleave` 并行读取多个分片文件

### 参考文档

- [tf.data 性能优化指南](https://www.tensorflow.org/guide/data_performance)
- [tf.data.Dataset API](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)